---
# Lab Task: Create a Yield Curve Hedge Method #
---

In this lab, you will extend the `YieldCurve` class to add a sophisticated hedging capability that calculates hedge positions to offset interest rate risk.

## Background: Interest Rate Risk and Hedging

When holding fixed-income positions, changes in interest rates affect their value. **Hedging** is the practice of taking offsetting positions to reduce this risk.

### Key Concepts:

**Delta Hedging**: Calculating sensitivity to small changes in market rates and taking offsetting positions

**Yield Curve Risk**: Different instruments are sensitive to different parts of the yield curve (short-term vs long-term rates)

**Hedge Portfolio**: A portfolio of liquid instruments (the reference instruments used to build the curve) that can offset the risk of a position

### The Bumping Method

To calculate hedge ratios, we use a technique called **bumping**:

1. Calculate the base NPV of the position using the current yield curve
2. For each instrument in the reference portfolio:
   - Bump its yield by a small amount (e.g., 1 basis point = 0.0001)
   - Recalculate the cash flows for that instrument
   - Rebuild the yield curve using the bootstrap method
   - Recalculate the NPV of the position
   - Calculate the change in NPV: $\Delta NPV = NPV_{bumped} - NPV_{base}$
3. The hedge ratio for each instrument is: 
   $$\text{Hedge Ratio} = -\frac{\Delta NPV_{position}}{\Delta NPV_{hedge\_instrument}}$$

### Why This Works

- Each reference instrument affects a specific part of the yield curve
- By bumping each instrument, we measure how sensitive our position is to changes in different maturities
- The hedge ratios tell us how much of each reference instrument to hold to offset our position's risk

## Lab Tasks

You will create a new method `calculate_hedge()` for the `YieldCurve` class that:
1. Takes a position (any instrument object) as input
2. Calculates hedge ratios for each instrument in the reference portfolio
3. Returns a hedge portfolio with appropriate positions

---

### Step 0: Import Pre-existing Classes

Import the necessary classes for working with instruments and yield curves.

In [ ]:
from instrument_classes import Bank_bill, Bond, Portfolio
from curve_classes_and_functions import ZeroCurve, YieldCurve
import pandas as pd
import copy

---

### Step 1: Create Reference Portfolio Instruments

Create a set of bank bills and bonds that will form the reference portfolio used to construct the yield curve.

**Task**: Create at least:
- 2-3 bank bills at different maturities (e.g., 0.25, 0.5 years)
- 2-3 bonds at longer maturities (e.g., 1, 2, 3 years)

Make sure to set all required attributes (face value, maturity, coupon, frequency, YTM) and generate cash flows.

In [ ]:
# Example structure:
# bill_1 = Bank_bill(face_value=100, maturity=0.25)
# bill_1.set_ytm(0.03)
# bill_1.set_cash_flows()

# Your code here:


---

### Step 2: Build the Reference Portfolio and Yield Curve

**Task**: 
1. Create a Portfolio object and add your bills and bonds to it
2. Create a YieldCurve object
3. Set the constituent portfolio
4. Bootstrap the yield curve
5. Print the resulting zero curve to verify it was constructed correctly

In [ ]:
# Example structure:
# ref_portfolio = Portfolio()
# ref_portfolio.add_bank_bill(bill_1)
# ref_portfolio.add_bond(bond_1)
# ...

# yc = YieldCurve()
# yc.set_constituent_portfolio(ref_portfolio)
# yc.bootstrap()

# Your code here:


---

### Step 3: Create a Position to Hedge

Create an instrument position that you want to hedge. This could be:
- A single bond with different characteristics than the reference bonds
- A portfolio of instruments
- Any instrument object with cash flows

**Task**: Create a position and calculate its NPV using your yield curve.

In [ ]:
# Example: Create a bond position to hedge
# position = Bond()
# position.set_face_value(1000)
# position.set_maturity(2.5)
# position.set_coupon(0.05)
# position.set_frequency(2)
# position.set_ytm(0.045)
# position.set_cash_flows()

# position_npv = yc.npv(position)
# print(f"Position NPV: ${position_npv:.2f}")

# Your code here:


---

### Step 4: Design the calculate_hedge() Method

This is the main challenge! You need to create a method for the `YieldCurve` class that calculates hedge ratios.

**Method Signature**:
```python
def calculate_hedge(self, position, bump_size=0.0001):
    """
    Calculate hedge positions for a given position.
    
    Args:
        position: An instrument object (Bond, Bank_bill, Portfolio, etc.)
        bump_size: The yield bump size (default 1 basis point = 0.0001)
    
    Returns:
        hedge_portfolio: A Portfolio object with hedge positions
        hedge_details: A dictionary with detailed hedge information
    """
```

**Algorithm**:

1. Calculate base NPV of the position using the current curve

2. For each bank bill in the reference portfolio:
   - Create a copy of the bill
   - Bump its YTM by bump_size
   - Recalculate its cash flows
   - Create a new YieldCurve with this bumped bill (keeping other instruments the same)
   - Bootstrap the new curve
   - Calculate new NPV of the position
   - Calculate delta NPV: `delta_npv = new_npv - base_npv`
   - Also calculate the change in value of the bumped bill itself
   - Calculate hedge ratio: `-delta_npv_position / delta_value_bill`

3. Repeat for each bond in the reference portfolio

4. Create a Portfolio with the hedge positions and return it

**Tips**:
- Use `copy.deepcopy()` to create independent copies of instruments
- You'll need to access `self.portfolio.get_bank_bills()` and `self.portfolio.get_bonds()`
- The hedge ratio is negative because you want an offsetting position
- Store both the hedge portfolio and detailed information (sensitivities, ratios) to return

**Challenge**: Think about how to handle the fact that bumping one instrument affects the entire curve!

In [ ]:
# Create a new class that extends YieldCurve
class YieldCurveWithHedge(YieldCurve):
    """
    Extends YieldCurve to add hedging capabilities.
    """
    
    def calculate_hedge(self, position, bump_size=0.0001):
        """
        Calculate hedge positions for a given position.
        
        Your implementation here.
        """
        # Your code here
        pass

# Your implementation here:


---

### Step 5: Test Your Hedge Method

**Task**:
1. Create an instance of your `YieldCurveWithHedge` class
2. Set the constituent portfolio and bootstrap
3. Call `calculate_hedge()` on your position
4. Display the hedge ratios in a clear format (DataFrame recommended)
5. Verify the hedge by:
   - Bumping the entire yield curve
   - Calculating the P&L on your position
   - Calculating the P&L on your hedge portfolio
   - Showing that they approximately offset

**Expected Output**: A table showing:
- Instrument (bill/bond identifier)
- Maturity
- Sensitivity (delta NPV per basis point)
- Hedge ratio (position size needed)
- Hedge notional amount

In [ ]:
# Test your implementation
# Your code here:


---

### Step 6 (Optional Challenge): Verify the Hedge

Create a verification function that:
1. Bumps all yields in the reference portfolio by a larger amount (e.g., 10 basis points)
2. Calculates the P&L on the original position
3. Calculates the P&L on the hedge portfolio (with the calculated hedge ratios)
4. Shows that the combined P&L is close to zero

This demonstrates that your hedge is working correctly!

In [ ]:
# Verification code
# Your code here:


---

## Reflection Questions

After completing this lab, consider:

1. **Sensitivity Analysis**: Why does bumping different instruments have different impacts on your position's NPV? How does this relate to duration and key rate duration?

2. **Hedge Effectiveness**: Is your hedge perfect? Why or why not? What factors might cause the hedge to be imperfect?

3. **Practical Considerations**: In real markets, you can't always trade exact fractional positions. How would you handle this? What if transaction costs are significant?

4. **Curve Shape**: How would your hedge ratios change if the yield curve shape changes (e.g., from upward sloping to inverted)?

5. **Dynamic Hedging**: Hedge ratios change as time passes and yields move. How often would you need to rebalance your hedge in practice?

6. **OOP Benefits**: How does using the YieldCurve class make this hedging calculation easier? What would change if you had to manage all the yield curve data manually?